In [3]:
import torch
import torch.nn as nn, optim
import numpy as np
import cv2 # pip install opencv-python
import torch.nn.functional as F

In [ ]:
class GradCAM(nn.Module):
    def __init__(self, net, cls) -> None:
        super().__init__()
        self.net = net
        self.net.zero_grad()
        self.cls = cls
        self.act = {}
        self.grad = {}
    def backward_hook(self, net, inp, out):
        self.grad['grad'] = out
        return 
    def forward_hook(self, net, inp, out):
        self.act['act'] = out
        return 
    def get_gradcam(self, img, layer, org_shape):
        layer.register_forward_hook(self.forward_hook)
        layer.register_backward_hook(self.backward_hook)
        self.net.train() # net.eval() <- grad 비활성
        pred = self.net(img)
        score = pred[:, self.cls].squeeze()
        score.backward()
        grad = self.grad['grad']
        acti = self.act['act']
        b, c, u, v = grad.size()
        alpha = grad.view(b,c,-1).mean(2)
        alpha = alpha.view(b,c,1,1)
        cam = F.relu((alpha * acti).sum(1, keepdim=True)).detach().numpy().squeeze()
        cam = cv2.resize(cam, org_shape, cv2.INTER_LINEAR)
        return cam